# 🧠 Feedback迎合性分析 - Google Colab版

このノートブックは、Google Colab環境でFeedbackデータセットを使用してLLMの迎合性（Sycophancy）分析を実行します。

## 🎯 特徴
- **Gemma-2-27B対応**: 大規模モデルでの詳細分析
- **SAE内部状態抽出**: Layer 31のSAE活性化を詳細に記録
- **最適な分析位置**: プロンプトの最後のトークン（応答生成直前）の内部状態を取得
- **テンプレート比較**: 5種類のプロンプトテンプレートでの応答を比較
- **A100 GPU最適化**: Google Colab Pro A100環境での最適パフォーマンス

## 📋 実行順序
1. **環境セットアップ**: Google Drive マウント & 依存関係インストール
2. **実験設定**: サンプル数、トークン保存設定などをカスタマイズ
3. **分析実行**: Feedback迎合性分析の実行
4. **結果確認**: 分析結果とSAE活性化の確認

## 💡 分析位置について
迎合性分析では、**プロンプトの最後のトークン**（応答の最初のトークン生成直前）の内部状態を取得します。
これは、モデルがプロンプト全体を処理し終えた直後の状態で、「ユーザーに迎合する」という応答方針が
最も明確に表現されていると考えられます。

---

**⚠️ 重要**: このノートブックはGoogle Colab環境での使用を前提としています。

## 🔧 環境セットアップ (Google Drive マウント版)

In [ ]:
# 🔧 環境セットアップ (Google Drive マウント版)
# =================================================
import os
import sys
import torch

# 1. Google Driveをマウント
print("🔄 Google Driveをマウント中...")
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Driveマウント完了")
except Exception as e:
    print(f"❌ Google Driveマウントエラー: {e}")
    raise

# 2. プロジェクトのパスを設定
# ※Google Drive内のプロジェクトフォルダパスを指定してください
PROJECT_PATH = '/content/drive/MyDrive/sae_pj2'

if not os.path.exists(PROJECT_PATH):
    print(f"❌ プロジェクトフォルダが見つかりません: {PROJECT_PATH}")
    print("💡 PROJECT_PATHの変数が正しいか、フォルダが同期されているか確認してください。")
else:
    print(f"✅ プロジェクトフォルダ発見: {PROJECT_PATH}")

    # 3. Pythonがプロジェクト内の.pyファイルを読み込めるようにパスを追加
    if PROJECT_PATH not in sys.path:
        sys.path.insert(0, PROJECT_PATH)
        print(f"   🐍 Pythonパスに追加しました")

    # colabフォルダもパスに追加
    colab_path = os.path.join(PROJECT_PATH, 'colab')
    if colab_path not in sys.path:
        sys.path.insert(0, colab_path)
        print(f"   🐍 colabパスに追加しました")

    # 4. 作業ディレクトリをプロジェクトフォルダに移動
    os.chdir(PROJECT_PATH)
    print(f"   📁 作業ディレクトリを変更しました: {os.getcwd()}")

    # 4.5 必須ファイルの存在チェック
    required_items = [
        ('config.py', '設定ファイル'),
        ('colab/feedback_analyzer.py', 'メインロジック'),
        ('eval_dataset', 'データセットフォルダ'),
        ('eval_dataset/feedback.jsonl', 'Feedbackデータ')
    ]
    for path, label in required_items:
        exists = os.path.exists(path)
        icon = '✅' if exists else '⚠️'
        kind = '📄' if os.path.isfile(path) else '📁'
        print(f"   {icon} 必須確認: {kind} {path} - {label}{'' if exists else '（見つからない場合は同期を確認）'}")

# 5. 依存関係のインストール
print("\n📦 依存関係をインストール中...")
req_file = 'requirements-colab.txt'
if os.path.exists(req_file):
    try:
        get_ipython().system(f"pip install -q -r {req_file}")
        print("✅ 依存関係インストール完了 (requirements-colab.txtを使用)")
    except Exception as e:
        print(f"⚠️ 依存関係インストールで警告/エラー: {e}")
        print("   必要に応じて個別に !pip install <package> を実行してください")
else:
    print(f"⚠️ {req_file} が見つかりません。必要なライブラリが不足する可能性があります。")

# 6. 環境確認
print("\n🚀 環境セットアップ完了！")
if torch.cuda.is_available():
    try:
        print(f"   GPU検出: {torch.cuda.get_device_name(0)}")
        memory_total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"   GPU Memory: {memory_total:.1f}GB")
    except Exception:
        print("   GPU検出: 利用可能 (デバイス名の取得に失敗)")
else:
    print("   ❌ GPU利用不可 (CPUモード)")


## 📚 メインコードのインポートと設定

In [ ]:
# 📚 メインコードのインポートと設定
try:
    # メインの分析クラスをインポート
    from feedback_analyzer import FeedbackAnalyzer

    # 設定をインポート
    from config import (
        FEEDBACK_GEMMA27B_CONFIG,
        FEEDBACK_GEMMA2_9B_IT_CONFIG,
        FeedbackConfig
    )

    print("✅ メインコードのインポート完了")
    print("📋 利用可能な設定:")
    print("   🔸 FEEDBACK_GEMMA2_9B_IT_CONFIG - Gemma-2-9B-it Feedback分析用")
    print("   🔸 FEEDBACK_GEMMA27B_CONFIG - Gemma-2-27B Feedback分析用")

except ImportError as e:
    print(f"❌ インポートエラー: {e}")
    print("💡 まず環境セットアップセルを実行し、PROJECT_PATHが正しいことを確認してください。")
    try:
        proj = PROJECT_PATH if 'PROJECT_PATH' in globals() else '(未定義)'
        print(f"   PROJECT_PATH: {proj}")
        cwd = os.getcwd()
        print(f"   CWD: {cwd}")
        print("   🔎 CWD 内のファイル一覧:")
        print("      ", ', '.join(os.listdir(cwd)[:20]))
    except Exception as ee:
        print(f"   追加診断中のエラー: {ee}")
    raise

# メモリ最適化設定
import warnings
warnings.filterwarnings('ignore')
os.environ['TOKENIZERS_PARALLELISM'] = 'false'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

print(f"\n✅ 設定完了 - 次のセルで実験パラメータを設定してください")


## ⚙️ 実験パラメータの設定

ここで実験の詳細設定を行います。サンプル数、トークン保存設定などをカスタマイズできます。

**重要**: デフォルト設定（`SAVE_ALL_TOKENS = False`）では、プロンプトの最後のトークン（応答生成直前）の内部状態のみを取得します。これが迎合性分析に最適な設定です。

In [ ]:
# ⚙️ 実験パラメータの設定
# =================================================
from copy import deepcopy

# ベース設定を取得
EXPERIMENT_CONFIG = deepcopy(FEEDBACK_GEMMA2_9B_IT_CONFIG)

# 🎯 実験パラメータ（ここを変更して実験条件をカスタマイズ）
# ===============================================

# 1. サンプル数設定
# SAMPLE_SIZE = 100 # 分析する質問数（1問あたり5つのテンプレートバリエーション）
start = 1189
end = 1200
# 2. SAE内部状態の保存設定
# 重要: False（推奨）= プロンプトの最後のトークン（応答生成直前）のみ
#       True = 全プロンプトトークンの活性化を保存
SAVE_ALL_TOKENS = False

# 3. 生成パラメータ
MAX_NEW_TOKENS = 512  # 生成する最大トークン数
TEMPERATURE = 0.7     # 生成温度（0.0-1.0、低いほど決定的）

# 4. デバッグ設定
VERBOSE = True        # 詳細ログを表示
SHOW_PROMPTS = True   # プロンプトを表示
SHOW_RESPONSES = True # 応答を表示

# 設定を適用
# EXPERIMENT_CONFIG.data.sample_size = SAMPLE_SIZE
EXPERIMENT_CONFIG.generation.max_new_tokens = MAX_NEW_TOKENS
EXPERIMENT_CONFIG.generation.temperature = TEMPERATURE
EXPERIMENT_CONFIG.debug.verbose = VERBOSE
EXPERIMENT_CONFIG.debug.show_prompts = SHOW_PROMPTS
EXPERIMENT_CONFIG.debug.show_responses = SHOW_RESPONSES

# Feedback専用設定を適用
if not hasattr(EXPERIMENT_CONFIG, 'feedback'):
    EXPERIMENT_CONFIG.feedback = FeedbackConfig()
EXPERIMENT_CONFIG.feedback.save_all_tokens = SAVE_ALL_TOKENS

# 設定サマリーを表示
print("🎯 実験設定:")
print(f"   📱 モデル: {EXPERIMENT_CONFIG.model.name}")
# print(f"   🧪 サンプル数: {EXPERIMENT_CONFIG.data.sample_size} 問題")
print(f"   🎱 Start Question ID:{start}, End Question ID: {end}")
print(f"   📊 テンプレート数: 5種類/問題")
print(f"   💾 分析位置: {'全プロンプトトークン' if SAVE_ALL_TOKENS else 'プロンプト最終トークン（応答生成直前）'}")
print(f"   🎯 対象レイヤー: {EXPERIMENT_CONFIG.feedback.target_layer}")
print(f"   🌡️  温度: {EXPERIMENT_CONFIG.generation.temperature}")
print(f"   🔢 最大トークン: {EXPERIMENT_CONFIG.generation.max_new_tokens}")
print(f"   🔍 詳細ログ: {EXPERIMENT_CONFIG.debug.verbose}")
print(f"   🔍 SAE: {EXPERIMENT_CONFIG.model.sae_release}/{EXPERIMENT_CONFIG.model.sae_id}")

# データセットの存在チェック
ds_path = EXPERIMENT_CONFIG.data.dataset_path
if not os.path.exists(ds_path):
    print(f"\n⚠️ データセットが見つかりません: {ds_path}")
    # デフォルトパスを試行
    default_file = os.path.join('eval_dataset', 'feedback.jsonl')
    if os.path.exists(default_file):
        EXPERIMENT_CONFIG.data.dataset_path = default_file
        print(f"   ✅ 自動補正: dataset_path を {default_file} に変更しました")
    else:
        print("   ❌ eval_dataset/feedback.jsonl が見つかりません。ファイルを配置してください。")

print(f"\n✅ 実験パラメータ設定完了 - 次のセルで分析を実行してください")

## 🔬 Feedback迎合性分析の実行

モデルをロードし、Feedbackデータセットに対する分析を実行します。

In [ ]:
from huggingface_hub import login
login()  # 画面の指示に従いトークン入力

In [ ]:
# 🔬 Feedback迎合性分析の実行
print("🚀 Feedback迎合性分析を開始...")
print("=" * 60)

# 実験メタデータの準備
import datetime
experiment_metadata = {
    'experiment_start': datetime.datetime.now().isoformat(),
    'config_params': {
        'model_name': EXPERIMENT_CONFIG.model.name,
        # 'sample_size': EXPERIMENT_CONFIG.data.sample_size,
        'save_all_tokens': EXPERIMENT_CONFIG.feedback.save_all_tokens,
        'target_layer': EXPERIMENT_CONFIG.feedback.target_layer,
        'temperature': EXPERIMENT_CONFIG.generation.temperature,
        'max_new_tokens': EXPERIMENT_CONFIG.generation.max_new_tokens,
    }
}

print(f"📋 実験メタデータ:")
print(f"   ⏰ 開始時刻: {experiment_metadata['experiment_start']}")
print(f"   📊 設定: {experiment_metadata['config_params']}")

try:
    # 分析器の初期化
    print(f"\n🔧 分析器を初期化中...")
    analyzer = FeedbackAnalyzer(EXPERIMENT_CONFIG)

    # メモリ使用状況の表示
    if torch.cuda.is_available():
        memory_used = torch.cuda.memory_allocated(0) / 1024**3
        memory_total = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"💾 初期GPU Memory: {memory_used:.1f}GB / {memory_total:.1f}GB")
        experiment_metadata['initial_memory_gb'] = round(memory_used, 2)

    # 完全分析の実行（モデル読み込み + 分析 + 結果保存を一括実行）
    print(f"\n🔄 完全分析を実行中...")
    print(f"   📋 この処理には数分〜数十分かかる場合があります...")

    # 実行時間測定開始
    import time
    start_time = time.time()

    # メイン分析の実行
    analyzer.run_complete_analysis(start_index=start, end_index=end)

    # 実行時間測定終了
    end_time = time.time()
    execution_time = end_time - start_time
    experiment_metadata['execution_time_seconds'] = round(execution_time, 2)
    experiment_metadata['experiment_end'] = datetime.datetime.now().isoformat()

    print(f"\n" + "=" * 60)
    print("✅ 分析完了！")
    print(f"⏱️  実行時間: {execution_time:.1f}秒 ({execution_time/60:.1f}分)")

    # 結果の簡易表示
    if hasattr(analyzer, 'results') and analyzer.results:
        num_questions = len(analyzer.results)
        total_variations = sum(len(r.variations) for r in analyzer.results)

        # 実験メタデータに結果を追加
        experiment_metadata['results'] = {
            'num_questions': num_questions,
            'total_variations': total_variations,
        }

        print(f"\n📊 分析結果サマリー:")
        print(f"   📝 分析した問題数: {num_questions}")
        print(f"   📈 総バリエーション数: {total_variations}")
        print(f"   💾 トークン保存モード: {'全トークン' if EXPERIMENT_CONFIG.feedback.save_all_tokens else '最後のトークンのみ'}")

        # 最初の結果の概要を表示
        if num_questions > 0:
            first_result = analyzer.results[0]
            print(f"\n📄 最初の質問の概要:")
            print(f"   Dataset: {first_result.dataset}")
            print(f"   テンプレート数: {len(first_result.variations)}")
            for var in first_result.variations[:3]:  # 最初の3つだけ表示
                print(f"   - {var.prompt_info.prompt_template_type or '(base)'}: {len(var.response_text)} 文字")

    # メモリ使用状況の最終確認
    if torch.cuda.is_available():
        memory_used = torch.cuda.memory_allocated(0) / 1024**3
        print(f"\n💾 最終GPU Memory: {memory_used:.1f}GB / {memory_total:.1f}GB")
        experiment_metadata['final_memory_gb'] = round(memory_used, 2)

    # 実験ログの保存
    import json
    experiment_log_file = f"experiment_log_{experiment_metadata['experiment_start'][:19].replace(':', '-')}.json"
    with open(experiment_log_file, 'w', encoding='utf-8') as f:
        json.dump(experiment_metadata, f, indent=2, ensure_ascii=False)

    print(f"\n📋 実験ログ保存: {experiment_log_file}")
    print(f"💡 詳細な結果は次のセルで確認できます")

    # グローバル変数に結果を保存（可視化用）
    analysis_results = analyzer.results

except Exception as e:
    print(f"❌ 分析実行エラー: {e}")
    print(f"\n💡 トラブルシューティング:")
    print(f"   1. GPU メモリが不足していないか確認")
    print(f"   2. より小さなサンプルサイズの設定を試行")
    print(f"   3. ランタイムを再起動して再実行")

    # エラー情報を実験メタデータに記録
    experiment_metadata['error'] = str(e)
    experiment_metadata['status'] = 'failed'

    # エラー時のメモリクリア
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    raise

print(f"\n🎉 分析セクション完了 - 次のセルで結果を確認してください")


## 📊 分析結果の確認

分析結果とSAE活性化情報を確認します。

In [ ]:
# 📊 分析結果の確認
import pandas as pd
import json

# 結果データの確認
try:
    if 'analysis_results' in globals() and analysis_results:
        print("📈 分析結果の詳細を表示...")
        print("=" * 60)

        # 基本統計
        num_questions = len(analysis_results)
        total_variations = sum(len(r.variations) for r in analysis_results)

        print(f"📊 基本統計:")
        print(f"   📝 分析した問題数: {num_questions}")
        print(f"   📈 総バリエーション数: {total_variations}")
        print(f"   💾 平均バリエーション数/問題: {total_variations/num_questions:.1f}")

        # データセット別の統計
        dataset_counts = {}
        for result in analysis_results:
            dataset = result.dataset
            dataset_counts[dataset] = dataset_counts.get(dataset, 0) + 1

        print(f"\n📚 データセット別の内訳:")
        for dataset, count in dataset_counts.items():
            print(f"   - {dataset}: {count} 問題")

        # テンプレートタイプ別の統計
        template_stats = {}
        for result in analysis_results:
            for variation in result.variations:
                template_type = variation.prompt_info.prompt_template_type or "(base)"
                if template_type not in template_stats:
                    template_stats[template_type] = {
                        'count': 0,
                        'avg_response_length': 0,
                        'total_length': 0,
                        'avg_active_features': 0,
                        'total_features': 0
                    }

                stats = template_stats[template_type]
                stats['count'] += 1
                stats['total_length'] += len(variation.response_text)
                stats['total_features'] += len(variation.top_k_features)

        # 平均を計算
        for template_type, stats in template_stats.items():
            stats['avg_response_length'] = stats['total_length'] / stats['count']
            stats['avg_active_features'] = stats['total_features'] / stats['count']

        print(f"\n📝 テンプレートタイプ別の統計:")
        for template_type, stats in template_stats.items():
            print(f"   {template_type}:")
            print(f"      サンプル数: {stats['count']}")
            print(f"      平均応答長: {stats['avg_response_length']:.0f} 文字")
            print(f"      平均活性化特徴数: {stats['avg_active_features']:.1f}")

        # 具体例の表示（最初の2問）
        print(f"\n📝 具体例（最初の2問）:")
        print("-" * 60)

        for i, result in enumerate(analysis_results[:2]):
            print(f"\n問題 {i+1} (ID: {result.question_id}):")
            print(f"  データセット: {result.dataset}")
            print(f"  ベーステキスト: {result.base_text[:100]}...")
            print(f"  テンプレート数: {len(result.variations)}")

            for j, variation in enumerate(result.variations):
                template_type = variation.prompt_info.prompt_template_type or "(base)"
                print(f"\n  バリエーション {j+1}: {template_type}")
                print(f"    応答: {variation.response_text[:150]}...")
                print(f"    応答長: {len(variation.response_text)} 文字")
                print(f"    Top-3 SAE特徴:")
                for feat_id, feat_val in variation.top_k_features[:3]:
                    print(f"      Feature {feat_id}: {feat_val:.4f}")
                print(f"    生成時間: {variation.metadata.get('generation_time_ms', 0):.0f} ms")

        # DataFrameに変換して保存
        print(f"\n💾 結果をDataFrameに変換中...")

        rows = []
        for result in analysis_results:
            for variation in result.variations:
                row = {
                    'question_id': result.question_id,
                    'dataset': result.dataset,
                    'template_type': variation.prompt_info.prompt_template_type or "(base)",
                    'response_length': len(variation.response_text),
                    'num_active_features': len(variation.top_k_features),
                    'top_feature_id': variation.top_k_features[0][0] if variation.top_k_features else None,
                    'top_feature_value': variation.top_k_features[0][1] if variation.top_k_features else None,
                    'generation_time_ms': variation.metadata.get('generation_time_ms', 0)
                }
                rows.append(row)

        df_results = pd.DataFrame(rows)

        print(f"✅ DataFrame作成完了 ({len(df_results)} 行)")
        print(f"\nDataFrame サンプル:")
        print(df_results.head(10))

        # CSVに保存
        csv_path = f"results/feedback/feedback_analysis_summary.csv"
        os.makedirs("results/feedback", exist_ok=True)
        df_results.to_csv(csv_path, index=False)
        print(f"\n💾 結果を保存しました: {csv_path}")

        print(f"\n" + "=" * 60)
        print("✅ 結果確認完了！")

    else:
        print("❌ 分析結果が見つかりません")
        print("💡 上記の分析実行セルを先に実行してください")

except Exception as e:
    print(f"❌ 結果表示エラー: {e}")
    import traceback
    traceback.print_exc()
    print("💡 分析が正常に完了しているか確認してください")


## 📈 SAE活性化の可視化（オプション）

Top-k SAE特徴の分布を可視化します。

In [ ]:
# 📈 SAE活性化の可視化（オプション）
try:
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import numpy as np

    if 'analysis_results' in globals() and analysis_results:
        print("📊 SAE活性化の可視化を開始...")

        # 1. テンプレートタイプ別の平均活性化値
        template_activations = {}

        for result in analysis_results:
            for variation in result.variations:
                template_type = variation.prompt_info.prompt_template_type or "(base)"
                if template_type not in template_activations:
                    template_activations[template_type] = []

                # Top-k特徴の活性化値を収集
                for feat_id, feat_val in variation.top_k_features:
                    template_activations[template_type].append(feat_val)

        # 箱ひげ図を作成
        fig1 = go.Figure()

        for template_type, activations in template_activations.items():
            fig1.add_trace(go.Box(
                y=activations,
                name=template_type,
                boxmean='sd'
            ))

        fig1.update_layout(
            title="テンプレートタイプ別のSAE活性化値分布",
            yaxis_title="活性化値",
            xaxis_title="テンプレートタイプ",
            height=500
        )

        fig1.show()

        # 2. 最も活性化した特徴の頻度
        feature_counts = {}

        for result in analysis_results:
            for variation in result.variations:
                if variation.top_k_features:
                    top_feature_id = variation.top_k_features[0][0]
                    feature_counts[top_feature_id] = feature_counts.get(top_feature_id, 0) + 1

        # 頻度上位20個を可視化
        sorted_features = sorted(feature_counts.items(), key=lambda x: x[1], reverse=True)[:20]
        feature_ids = [f"Feature {fid}" for fid, _ in sorted_features]
        counts = [count for _, count in sorted_features]

        fig2 = go.Figure(data=[
            go.Bar(x=feature_ids, y=counts, text=counts, textposition='auto')
        ])

        fig2.update_layout(
            title="最も活性化したSAE特徴（Top-20）",
            xaxis_title="特徴ID",
            yaxis_title="出現回数",
            height=500
        )

        fig2.show()

        # 3. テンプレートタイプ別の応答長
        template_lengths = {t: [] for t in template_activations.keys()}

        for result in analysis_results:
            for variation in result.variations:
                template_type = variation.prompt_info.prompt_template_type or "(base)"
                template_lengths[template_type].append(len(variation.response_text))

        fig3 = go.Figure()

        for template_type, lengths in template_lengths.items():
            fig3.add_trace(go.Box(
                y=lengths,
                name=template_type,
                boxmean='sd'
            ))

        fig3.update_layout(
            title="テンプレートタイプ別の応答長分布",
            yaxis_title="応答長（文字数）",
            xaxis_title="テンプレートタイプ",
            height=500
        )

        fig3.show()

        print("\n✅ 可視化完了！")

    else:
        print("❌ 分析結果が見つかりません")
        print("💡 上記の分析実行セルを先に実行してください")

except ImportError:
    print("⚠️ plotlyがインストールされていません")
    print("!pip install plotly を実行してください")
except Exception as e:
    print(f"❌ 可視化エラー: {e}")
    import traceback
    traceback.print_exc()


## 🔍 個別の質問の詳細分析

特定の質問についてテンプレート間のSAE活性化を比較します。

In [ ]:
# 🔍 個別の質問の詳細分析

# 分析したい質問のIDを指定（0から始まる）
QUESTION_ID_TO_ANALYZE = 1

try:
    if 'analysis_results' in globals() and analysis_results:
        if QUESTION_ID_TO_ANALYZE < len(analysis_results):
            result = analysis_results[QUESTION_ID_TO_ANALYZE]

            print(f"🔍 質問 {QUESTION_ID_TO_ANALYZE} の詳細分析")
            print("=" * 60)
            print(f"Dataset: {result.dataset}")
            print(f"ベーステキスト: {result.base_text[:200]}...")
            print(f"\nテンプレートバリエーション数: {len(result.variations)}")

            # 各バリエーションの詳細
            for i, variation in enumerate(result.variations):
                template_type = variation.prompt_info.prompt_template_type or "(base)"
                print(f"\n{'='*60}")
                print(f"バリエーション {i+1}: {template_type}")
                print(f"{'='*60}")

                if EXPERIMENT_CONFIG.debug.show_prompts:
                    print(f"\n📝 プロンプト:")
                    print(f"{variation.prompt_info.prompt}")

                print(f"\n💬 応答:")
                print(f"{variation.response_text}")

                print(f"\n📊 統計:")
                print(f"  応答長: {len(variation.response_text)} 文字")
                print(f"  生成時間: {variation.metadata.get('generation_time_ms', 0):.0f} ms")

                print(f"\n🧠 Top-10 SAE特徴:")
                for j, (feat_id, feat_val) in enumerate(variation.top_k_features[:10], 1):
                    print(f"  {j:2d}. Feature {feat_id:5d}: {feat_val:.4f}")

                # SAE活性化の詳細統計
                if variation.sae_activations:
                    num_tokens = len(variation.sae_activations) if EXPERIMENT_CONFIG.feedback.save_all_tokens else 1
                    print(f"\n💾 SAE活性化情報:")
                    print(f"  保存されたトークン数: {num_tokens}")

                    if EXPERIMENT_CONFIG.feedback.save_all_tokens:
                        # 各トークンの活性化特徴数
                        for token_key, features in list(variation.sae_activations.items())[:5]:
                            print(f"  {token_key}: {len(features)} 活性化特徴")
                    else:
                        last_token_features = variation.sae_activations.get("last_token", {})
                        print(f"  最後のトークンの活性化特徴数: {len(last_token_features)}")

            # テンプレート間のSAE特徴比較
            print(f"\n{'='*60}")
            print("📊 テンプレート間のTop特徴比較")
            print(f"{'='*60}")

            # 各テンプレートのTop-1特徴を比較
            print(f"\n{'テンプレート':<30} {'Top特徴ID':<12} {'活性化値':<10}")
            print("-" * 60)
            for variation in result.variations:
                template_type = variation.prompt_info.prompt_template_type or "(base)"
                if variation.top_k_features:
                    top_feat_id, top_feat_val = variation.top_k_features[0]
                    print(f"{template_type:<30} {top_feat_id:<12} {top_feat_val:<10.4f}")

            # 共通して活性化した特徴を探す
            print(f"\n🔄 全テンプレートで共通して活性化した特徴:")
            all_feature_sets = []
            for variation in result.variations:
                feature_set = set([feat_id for feat_id, _ in variation.top_k_features[:5]])
                all_feature_sets.append(feature_set)

            if all_feature_sets:
                common_features = set.intersection(*all_feature_sets)
                if common_features:
                    print(f"  共通特徴: {', '.join([f'Feature {fid}' for fid in sorted(common_features)])}")
                else:
                    print(f"  共通特徴はありません（Top-5の範囲内）")

        else:
            print(f"❌ 質問ID {QUESTION_ID_TO_ANALYZE} が範囲外です")
            print(f"💡 0〜{len(analysis_results)-1} の範囲で指定してください")
    else:
        print("❌ 分析結果が見つかりません")
        print("💡 上記の分析実行セルを先に実行してください")

except Exception as e:
    print(f"❌ エラー: {e}")
    import traceback
    traceback.print_exc()
